In [0]:
import numpy as np
import matplotlib.pyplot as plt

In [0]:
# generating images with one channel of pixel values in range 0-255
# params :
# (1) image_height : height of image
# (2) image_width  : width of image
# (3) batch_size   : number of images in the batch sample

def generate_batch_images(image_height=28, 
                          image_width=28, 
                          batch_size=50) :
  
  batch_images = np.zeros((batch_size, image_height, image_width))
  
  for i in range(batch_size) :
    batch_images[i] = np.random.randint(0, 
                                        high=256, 
                                        size=(image_height, image_width))
  
  return batch_images

In [0]:
# applying zero padding to the image
# params : 
# (1) image : numpy array image of one channel
# (2) padding_size_arr : tuple of tuples containing 
#                        the description for padding

def apply_zero_padding_image(image, padding_size_arr) :
  assert(len(padding_size_arr) == len(image.shape))
  image = np.pad(image, padding_size_arr, 'constant')
  
  return image

In [0]:
# apply zero padding to batch of images
# params : 
# (1) batch : batch of numpy images of one channel
# (2) padding_size_arr : tuple of tuples containing 
#                        the description for padding

def apply_zero_padding_batch(batch, padding_size_arr) :
  
  assert(len(padding_size_arr) == len(batch.shape) - 1)
    
  new_size = [batch.shape[0]]
  for i in range(0, len(padding_size_arr)) :
    new_size.append(batch.shape[i + 1] + 2 * padding_size_arr[i][0])
  new_size = tuple(new_size)
    
  new_batch = np.zeros(new_size)
  
  for i in range(batch.shape[0]) :
    new_batch[i] = apply_zero_padding_image(batch[i], padding_size_arr)
    
  return new_batch

In [158]:
#(Q1) applying padding

padding_size_arr = ((1, 1), (2, 2))
batch = generate_batch_images()
print("old shape => ", batch.shape)
new_batch = apply_zero_padding_batch(batch, padding_size_arr)
print("new shape => ", new_batch.shape)

old shape =>  (50, 28, 28)
new shape =>  (50, 30, 32)


In [0]:
# convolution at one position, here applied at beginning
# params :
# (1) image: numpy array image in one channel
# (2) convolution filter(usually of size 3 X 3)

def convolve_at_beginning(image, convolve_filter) :
  
  assert(convolve_filter.shape[0] <= image.shape[0]
     and convolve_filter.shape[1] <= image.shape[1])
  
  result = 0
  
  for i in range(convolve_filter.shape[0]) :
    for j in range(convolve_filter.shape[1]) :
      result += convolve_filter[i][j] * image[i][j]
      
  return result

In [160]:
#(Q2) applying convolution operation at beginning

image = batch[0]
convolve_filter = np.array([[1, 1, 1], [0, 0, 0], [0, 0, 0]])

convolution_result = convolve_at_beginning(image, convolve_filter)
print("result of convolution is => ", convolution_result)

result of convolution is =>  412.0


In [0]:
convolve_filters = np.array([
    [[1, 1, 1],
    [0, 0, 0],
    [0, 0, 0]],
    
    [[0, 0, 0],
    [1, 1, 1],
    [0, 0, 0]],
    
    [[0, 0, 0],
    [0, 0, 0],
    [1, 1, 1]],
    
    [[1, 1, 1],
    [0, 0, 0],
    [-1, -1, -1]],
    
    [[1, 0, -1],
    [1, 0, -1],
    [1, 0, -1]]
])

bias_convolve_filters = np.array([5, 4, 3, 2, 1])

In [0]:
def activation_function_relu(image) :
  return np.maximum(image, 0)

def activation_function_leaky_relu(image) :
  return np.maximum(image, 0.01 * image)

In [0]:
# convolution for a list of filters, in this example
# I implemented using 3X3 simple filters
# params

# (1) A_prev : Activation from previous layer
# (2) convolve_filters : List of filters used for convolution
# (3) Stride : Stride used for convolution operation

def convolve_image_with_filters(A_prev, convolve_filters, bias_convolve_filters, stride=1) :
  
  filter_count = convolve_filters.shape[0]
  convolve_result_height = int(np.floor((A_prev.shape[0] - convolve_filters.shape[1])/stride + 1))
  convolve_result_width  = int(np.floor((A_prev.shape[1] - convolve_filters.shape[2])/stride + 1))
  
  convolve_result = np.zeros((filter_count, convolve_result_height, convolve_result_width))
  
  for i in range(convolve_result.shape[0]) :
    for j in range(convolve_result.shape[1]) :
      for k in range(convolve_result.shape[2]) :
        
        horiz_start = j * stride
        horiz_end = horiz_start + convolve_filters[i].shape[0]
        
        vert_start = k * stride
        vert_end = vert_start + convolve_filters[i].shape[1]
        
        image_section = A_prev[horiz_start : horiz_end, vert_start : vert_end]
        
        convolve_result[i][j][k] = np.sum(np.multiply(image_section, convolve_filters[i]))
    
    convolve_result[i] = convolve_result[i] + bias_convolve_filters[i]
    
    # apply activation function relu
    # convolve_result[i] = activation_function_relu(convolve_result[i])
    
  return convolve_result

In [164]:
# Applying convolution on image of one channel

convolution_result_1 = convolve_image_with_filters(image, convolve_filters, bias_convolve_filters, stride=2)
print("shape of convolution result => ", convolution_result_1.shape)

shape of convolution result =>  (5, 13, 13)


In [0]:
# average pooling on image of one channel
# params :

# (1) image : image of one channel on which pooling is applied
# (2) filter_size : filter size on which pooling is to be applied
# (3) stride : stride used for pooling operation

def average_pooling(image, filter_size, stride=1) :
  
  pool_result_height = int(np.floor((image.shape[0] - filter_size[0])/stride + 1))
  pool_result_width = int(np.floor((image.shape[1] - filter_size[1])/stride + 1))
  
  pool_result = np.zeros((pool_result_height, pool_result_width))
      
  for i in range(pool_result.shape[0]) :
    for j in range(pool_result.shape[1]) :
      
      horiz_start = i * stride
      horiz_end = horiz_start + filter_size[0]
      
      vert_start = j * stride
      vert_end = vert_start + filter_size[1]
      
      image_section = image[horiz_start : horiz_end, vert_start : vert_end]
      
      pool_result[i][j] = float(np.sum(image_section)/float(filter_size[0] * filter_size[1]))
      
  return pool_result

In [0]:
# max pooling on image of one channel
# params :

# (1) image : image of one channel on which pooling is applied
# (2) filter_size : filter size on which pooling is to be applied
# (3) stride : stride used for pooling operation

def max_pooling(image, filter_size, stride=1) :
  
  pool_result_height = int(np.floor((image.shape[0] - filter_size[0])/stride + 1))
  pool_result_width = int(np.floor((image.shape[1] - filter_size[1])/stride + 1))
  
  pool_result = np.zeros((pool_result_height, pool_result_width))
  
  for i in range(pool_result.shape[0]) :
    for j in range(pool_result.shape[1]) :
      
      horiz_start = i * stride
      horiz_end = horiz_start + filter_size[0]
      
      vert_start = j * stride
      vert_end = vert_start + filter_size[1]
      
      image_section = image[horiz_start : horiz_end, vert_start : vert_end]
      
      pool_result[i][j] = np.amax(image_section)
      
  return pool_result


In [0]:
# applying average pooling 

filter_size = np.array([3, 3])
average_pool_result = average_pooling(image, filter_size, stride=2)

# applying max pooling

max_pooling_result = max_pooling(image, filter_size, stride=2)

In CNN, the features are extracted by using the values present in the filter used for convolution and the bias corresponding to each filter. When we apply the convolution operation to the image and then use some activation function(like ReLU), we get some output. This output is propogated forward to many layers and in the end we apply backpropogation to adjust the values present in the CNN filter and bias.

The backpropogation algorithm is similar to that applied in DNN and we can use gradient descent to optimise the weights and bias terms.
